# FEATURE SELECTION

***

# FILTER METHODS

In [1]:
#libs

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold


ModuleNotFoundError: No module named 'numpy'